# SQLAlchemy Core

In [1]:
!pip install --upgrade sqlalchemy

Requirement already up-to-date: sqlalchemy in c:\users\ronan amicel\anaconda3\lib\site-packages (1.3.11)


## Établir une connexion

In [2]:
from sqlalchemy import create_engine

In [3]:
engine = create_engine("sqlite:///test.db")  # la base créée plus tôt via la DB API

In [34]:
# Pour MySQL :
# !pip install mysqlclient
# engine = create_engine("mysql+mysqldb://<username>:<password>@<host>:<port>/<dbname>")

In [5]:
# Pour MS SQL Server :
# !pip install pyodbc
# engine = create_engine("mssql+pyodbc://<username>:<password>@<dsnname>")"

In [6]:
engine

Engine(sqlite:///test.db)

In [7]:
conn = engine.connect()
conn

## Exécuter des requêtes

On peut utiliser cette connexion comme une connexion DB API pour faire des requêtes « en dur » :

In [8]:
cursor = conn.execute("SELECT * FROM fruits;")
cursor.fetchall()

[(1, 'banane'),
 (2, 'poire'),
 (3, 'framboise'),
 (4, 'banane'),
 (5, 'poire'),
 (6, 'framboise'),
 (7, 'pomme de reinette'),
 (8, "pomme d'api"),
 (9, 'banane'),
 (10, 'poire'),
 (11, 'framboise'),
 (12, 'pomme de reinette'),
 (13, "pomme d'api"),
 (14, "pomme d'api"),
 (15, 'ananas'),
 (16, 'clémentine'),
 (17, 'ananas'),
 (18, 'clémentine')]

## Utiliser SQLAlchemy Core pour construire des requêtes

**Avantage** : on exprime ses requêtes à plus haut niveau, et SQLAlchemy s'occupe de générer le code SQL en tenant compte du dialecte et des spécificités de la base de données utilisée.

In [9]:
engine = create_engine("sqlite:///:memory:", echo=True)

### Créer une table

On peut déclarer la structure de notre table :

In [10]:
from sqlalchemy import Table, Column, Integer, String, MetaData

metadata = MetaData()

fruits_table = Table(
    "fruits",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("nom", String),
)

Puis on peut générer les requêtes pour créer les tables déclarées :

In [11]:
metadata.create_all(engine)

2019-12-10 10:51:17,481 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-12-10 10:51:17,482 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 10:51:17,483 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-12-10 10:51:17,484 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 10:51:17,485 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("fruits")
2019-12-10 10:51:17,485 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 10:51:17,486 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("fruits")
2019-12-10 10:51:17,486 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 10:51:17,488 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE fruits (
	id INTEGER NOT NULL, 
	nom VARCHAR, 
	PRIMARY KEY (id)
)


2019-12-10 10:51:17,488 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 10:51:17,489 INFO sqlalchemy.engine.base.Engine COMMIT


### Insérer des données

On peut insérer une ligne dans la table :

In [12]:
conn = engine.connect()

In [13]:
query = fruits_table.insert().values(nom="banane")
conn.execute(query)

2019-12-10 10:51:17,501 INFO sqlalchemy.engine.base.Engine INSERT INTO fruits (nom) VALUES (?)
2019-12-10 10:51:17,502 INFO sqlalchemy.engine.base.Engine ('banane',)
2019-12-10 10:51:17,503 INFO sqlalchemy.engine.base.Engine COMMIT


On peut aussi insérer plusieurs lignes d'un coup :

In [14]:
fruit1 = {"nom": "pomme"}
query = fruits_table.insert()
conn.execute(query, [
    fruit1,
    {"nom": "kiwi"},
    {"nom": "framboise"},
])

2019-12-10 10:51:17,510 INFO sqlalchemy.engine.base.Engine INSERT INTO fruits (nom) VALUES (?)
2019-12-10 10:51:17,510 INFO sqlalchemy.engine.base.Engine (('pomme',), ('kiwi',), ('framboise',))
2019-12-10 10:51:17,511 INFO sqlalchemy.engine.base.Engine COMMIT


### Faire une requête

In [15]:
from sqlalchemy import select

query = select([fruits_table])
cursor = conn.execute(query)
cursor.fetchall()

2019-12-10 10:51:17,519 INFO sqlalchemy.engine.base.Engine SELECT fruits.id, fruits.nom 
FROM fruits
2019-12-10 10:51:17,520 INFO sqlalchemy.engine.base.Engine ()


[(1, 'banane'), (2, 'pomme'), (3, 'kiwi'), (4, 'framboise')]

On peut spécifier les colonnes voulues :

In [16]:
query = select([fruits_table.c.nom])
cursor = conn.execute(query)
cursor.fetchall()

2019-12-10 10:51:17,530 INFO sqlalchemy.engine.base.Engine SELECT fruits.nom 
FROM fruits
2019-12-10 10:51:17,531 INFO sqlalchemy.engine.base.Engine ()


[('banane',), ('pomme',), ('kiwi',), ('framboise',)]

On peut ajouter une clause `WHERE` :

In [17]:
query = select([fruits_table.c.nom]).where(fruits_table.c.nom.like("b%"))
cursor = conn.execute(query)
cursor.fetchall()

2019-12-10 10:51:17,541 INFO sqlalchemy.engine.base.Engine SELECT fruits.nom 
FROM fruits 
WHERE fruits.nom LIKE ?
2019-12-10 10:51:17,542 INFO sqlalchemy.engine.base.Engine ('b%',)


[('banane',)]

In [18]:
query = select([fruits_table.c.nom]).where(fruits_table.c.nom >= "pomme")
cursor = conn.execute(query)
cursor.fetchall()

2019-12-10 10:51:17,553 INFO sqlalchemy.engine.base.Engine SELECT fruits.nom 
FROM fruits 
WHERE fruits.nom >= ?
2019-12-10 10:51:17,554 INFO sqlalchemy.engine.base.Engine ('pomme',)


[('pomme',)]

In [19]:
from sqlalchemy import ForeignKey
b
barquette_table = Table(
    "barquettes",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("fruit_id", Integer, ForeignKey("fruits.id")),
    Column("poids", Integer),
)

metadata.create_all(engine)

2019-12-10 10:51:17,564 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("fruits")
2019-12-10 10:51:17,564 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 10:51:17,566 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("barquettes")
2019-12-10 10:51:17,566 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 10:51:17,567 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("barquettes")
2019-12-10 10:51:17,568 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 10:51:17,570 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE barquettes (
	id INTEGER NOT NULL, 
	fruit_id INTEGER, 
	poids INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(fruit_id) REFERENCES fruits (id)
)


2019-12-10 10:51:17,570 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 10:51:17,571 INFO sqlalchemy.engine.base.Engine COMMIT


In [23]:
framboise_id = conn.execute(select([fruits_table.c.id]).where(fruits_table.c.nom == "framboise")).fetchone()
framboise_id = framboise_id[0]

2019-12-10 11:07:06,680 INFO sqlalchemy.engine.base.Engine SELECT fruits.id 
FROM fruits 
WHERE fruits.nom = ?
2019-12-10 11:07:06,681 INFO sqlalchemy.engine.base.Engine ('framboise',)


In [24]:
query = barquette_table.insert().values(fruit_id=framboise_id, poids=500)
conn.execute(query)

2019-12-10 11:07:29,064 INFO sqlalchemy.engine.base.Engine INSERT INTO barquettes (fruit_id, poids) VALUES (?, ?)
2019-12-10 11:07:29,065 INFO sqlalchemy.engine.base.Engine (4, 500)
2019-12-10 11:07:29,066 INFO sqlalchemy.engine.base.Engine COMMIT


In [29]:
query = select([barquette_table.c.poids, fruits_table.c.nom]).select_from(
    fruits_table.join(
        barquette_table,
        barquette_table.c.fruit_id == fruits_table.c.id,
    )
)
cursor = conn.execute(query)
cursor.fetchall()

2019-12-10 11:11:59,041 INFO sqlalchemy.engine.base.Engine SELECT barquettes.poids, fruits.nom 
FROM fruits JOIN barquettes ON barquettes.fruit_id = fruits.id
2019-12-10 11:11:59,042 INFO sqlalchemy.engine.base.Engine ()


[(500, 'framboise')]

In [33]:
joined_tables = fruits_table.outerjoin(
    barquette_table,
    barquette_table.c.fruit_id == fruits_table.c.id,
)
query = select([barquette_table.c.poids, fruits_table.c.nom]).select_from(joined_tables)
cursor = conn.execute(query)
cursor.fetchall()

2019-12-10 11:15:18,408 INFO sqlalchemy.engine.base.Engine SELECT barquettes.poids, fruits.nom 
FROM fruits LEFT OUTER JOIN barquettes ON barquettes.fruit_id = fruits.id
2019-12-10 11:15:18,409 INFO sqlalchemy.engine.base.Engine ()


[(None, 'banane'), (None, 'pomme'), (None, 'kiwi'), (500, 'framboise')]

## Références

- https://docs.sqlalchemy.org/en/13/core/tutorial.html